## S&P Global - CORE2LOG: AUTOMATIC ROCK CLASSIFICATION
### Misael M. Morales, Oriyomi Raheem, Carlos Torres-Verdin, and Michael Pyrcz, UT-Austin; Murray Christie, Vladimir Rabinovich, S&P Global
#### 2024, Digital Reservoir Characterization Technology (DiReCT)
***

In [ ]:
from auto_rock_classification import *

In [ ]:
# RUN PROCESSING
# this module will load the Core data for all wells, and do:
# (1) filter the wells based on the minimum number of core points specified
# (2) apply a clustering technique to estimate the rock classes along the well
# (3) save the results of the predicted classes in a csv file for all wells
RockClassification(minpts=30, method='gmm', n_classes=3).run_processing()

In [ ]:
# RUN DASHBOARD
# this module will load the Core2Log Rock Classification dashboard:
# (1) specify the well to analyze
# (2) specify the rock classification technique to use
#   (2a) if the technique is physics-based: specify cutoffs
#   (2b) if the technique is ML-based: specify the number of classes
# (3) run the dashboard to visualize results
RockClassification(minpts=2, well_number=0, method='bisectkmeans', n_classes=3).run_dashboard(maketitle=False)

In [ ]:
# COMPARISON
# compare all techniques for a given well (user must define cutoffs and n_classes accordingly)
RockClassification(minpts=30, well_number=1104).run_comparison(n_classes=3, 
                                                    leverett_cutoffs=[10,20,40], 
                                                    winland_cutoffs=[150,300,500], 
                                                    lorenz_cutoffs=[0.5,2,5])

***
# END

In [4]:
import pyvista as pv
from auto_rock_classification import *

In [35]:
folder = 'Data/UT Export core classification'
files  = os.listdir(folder)[:100]

all_wells = {}
all_uwi   = []
for i, file in enumerate(files):
    df = pd.read_csv('{}/{}'.format(folder, file))

    uwi = df['UWI'].values[0]
    all_uwi.append(uwi)

    lat = df['SURFACE_LATITUDE'].values[0]
    lon = df['SURFACE_LONGITUDE'].values[0]

    d = df[['INTERVAL_DEPTH','CLASS']]

    x = {'LAT':lat, 'LON':lon, 'DAT':d}
    all_wells[uwi] = x

In [36]:
colors = ['dodgerblue', 'seagreen', 'firebrick', 'gold', 'black']
cmap2  = ListedColormap(colors[:len(np.unique(all_wells[all_uwi[0]]['DAT']['CLASS']))])